### Pair-approximation for the joint degree distribution

Assume we are selecting stubs for nodes sequentially. The first node, $v_1$, is a draw from a binomial degree distribution
\begin{align}
p(k_1=\kappa_1)=B_{n-1}^p(\kappa_1)\,,
\end{align}
where $B_{n-1}^p$ is a binomial distribution for $n-1$ draws each with a success rate $p$.
The second node, $v_2$, is either adjacent to $v_1$ or it is not. The remainder of the degree is a draw from a binomial distribution again.
\begin{align}
p(k_2=\kappa_2)=p(v_1\notin N(v_2))B^{n-2}_{p}(\kappa_2)+p(v_1\in N(v_2))B_{n-2}^p(\kappa_2-1)\,,
\end{align}
where $N(v_i)$ is the neighborhood of the node $v_i$. The probability of $v_1$ and $v_2$ being adjacent 
\begin{align}
p(v_1\in N(v_2)|k_1=\kappa_1) = \frac{\kappa_1}{n-1}\,.
\end{align}
This yields a conditional probability distribution for the degree of $v_2$:
\begin{align}
p(k_2=\kappa_2|k_1=\kappa_1)=\left(1-\frac{\kappa_1}{n-1}\right)B^{n-2}_p(\kappa_2)+\frac{\kappa_1}{n-1}B^{n-2}_p(\kappa_2-1)\,.
\end{align}
The corresponding joint probability distribution is
\begin{align}
p(k_1=\kappa_1,k_2=\kappa_2)=\left[\left(1-\frac{\kappa_1}{n-1}\right)B^{n-2}_p(\kappa_2)+\frac{\kappa_1}{n-1}B^n-2_p(\kappa_2-1)\right]B^{n-1}_p(\kappa_1)\,.
\end{align}
The third node, $v_3$ can be adjacent to $v_1$, $v_2$, both, or neither.
\begin{align}
p(k_3=\kappa_3)=&p(v_1,v_2\notin N(v_3))B^{n-3}_{p}(\kappa_3)\\
&+p(v_1\in N(v_3)\wedge v_2\notin N(v_3))B_{n-2}^p(\kappa_3-1)\nonumber\\
&+p(v_1\notin N(v_3)\wedge v_2\in N(v_3))B_{n-2}^p(\kappa_3-1)\nonumber\\
&+p(v_1,v2\in N(v_3))B_{n-2}^p(\kappa_3-2)\nonumber,\end{align}

For the $i$-th node, this leads to the conditional probability distribution
\begin{align}
p(k_i=\kappa_i)=\sum_{\ell=0}^i p(|V_{<i} \cap N(v_i)|=\ell | \kappa_1,\kappa_2,\dots)B^{n-i}_{p}(\kappa_3-\ell)\,,
\end{align}
where $V_{<i}:=\{v_1,v_2,\dots,v_{i-1}\}$. The probabilities $p(|V_{<i} \cap N(v_i)|=\ell)$ depend on the degrees of all nodes in $V_{<i}$ **and** how many of the edge stubs of these nodes have are linked to edge stubs of other nodes in $V_{<i}$. To simplify these probabilities, we are going to make a pair approximation. We assume that the probability $p(v_1\in N(v_i))$ is independent of $p(v_2\in N(v_i))$, $p(v_3\in N(v_i))$, and so on, even though it is not.
Using this approximation, we obtain
\begin{align}
p(|V_{<i} \cap N(v_i)|=i-1| \kappa_1,\kappa_2,\dots)=\frac{\kappa_1}{n-1}\frac{\kappa_2}{n-2}\dots \frac{\kappa_{i-1}}{n-(i-1)}
\end{align}
and
\begin{align}
p(|V_{<i} \cap N(v_i)|=\ell| \kappa_1,\kappa_2,\dots)=\sum_{V_{<i}[\ell]}
\left(\prod_{j\in V_{<i}[\ell]}\frac{\kappa_j}{n-j}\right)
\left(\prod_{j\in V_{<i}\backslash V_{<i}[\ell]}\left(1-\frac{\kappa_j}{n-j}\right)\right)\,,
\end{align}
where $V_{<i}[\ell]$ is a subset of $V_{<i}$ with $\ell$ elements.


In [1]:
import itertools
import math, time

def binomial_pmf(k, n, p):
    """Calculates the binomial probability mass function."""
    if k==0:
        return (1 - p) ** n
    elif k<0:
        return 0.0
    else:
        return math.comb(n, k) * (p ** k) * ((1 - p) ** (n - k))

def pair_approximation_probability(k_values, n, ell):
    """
    Computes the probability p(|V_{<i} ∩ N(v_i)| = ell) 
    using the pair approximation.

    k_values: List of degrees [k1, k2, ..., k_{i-1}]
    n: Total number of nodes
    ell: Number of neighbors in the intersection
    """
    V_i_minus_ell = itertools.combinations(range(len(k_values)), ell)

    probability_sum = 0

    for subset in V_i_minus_ell:
        prod_1 = 1
        prod_2 = 1
        for j in range(len(k_values)):
            if j in subset:
                prod_1 *= k_values[j] / (n - 1 - j)
            else:
                prod_2 *= (1 - k_values[j] / (n - 1 - j))
        
        probability_sum += prod_1 * prod_2

    return probability_sum

def p_k_i_given_degree_sequence(k_values, n, p, i, kappa_i):
    """
    Computes p(k_i = kappa_i) for a given degree sequence.

    k_values: List of degrees [k1, k2, ..., k_{i-1}]
    n: Total number of nodes
    p: Probability of edge between any two nodes
    i: Current node index
    kappa_i: Degree of the i-th node
    """
    probability = 0
    for ell in range(i):
        prob_ell = pair_approximation_probability(k_values[:i-1], n, ell)
        binomial_prob = binomial_pmf(kappa_i - ell, n - i, p)
        probability += prob_ell * binomial_prob
    
    return probability

def joint_probability_distribution(degree_sequence, n, p):
    """
    Computes the joint probability distribution for the entire degree sequence.

    degree_sequence: List of degrees [k1, k2, ..., k_n]
    n: Total number of nodes
    p: Probability of edge between any two nodes
    """
    joint_probability = 1.0
    
    for i in range(1, len(degree_sequence) + 1):
        kappa_i = degree_sequence[i-1]
        k_values = degree_sequence[:i-1]
        prob = p_k_i_given_degree_sequence(k_values, n, p, i, kappa_i)
        joint_probability *= prob
    
    return joint_probability

def expected_maximum_degree(n, p):
    """
    Calculates the expected maximum degree in a random graph.

    n: Total number of nodes
    p: Probability of edge between any two nodes
    """
    max_degree_expectation = 0
    possible_degrees = range(n)  # Possible degrees are from 0 to n-1

    for degree_sequence in itertools.product(possible_degrees, repeat=n):
        joint_prob = joint_probability_distribution(degree_sequence, n, p)
        max_degree = max(degree_sequence)
        max_degree_expectation += max_degree * joint_prob

    return max_degree_expectation

# Example usage
n = 10     # Example total number of nodes (small for computational feasibility)
p = 0.5   # Example probability of edge between any two nodes

t0=time.time()
expected_max_degree = expected_maximum_degree(n, p)
print(time.time()-t0)
print(f"Expected maximum degree in the random graph: {expected_max_degree:.4f}")
